Сегодня будем обрабатывать данные из Авито - https://www.kaggle.com/c/avito-context-ad-clicks .
В датасете присутствуют связанные между собой таблицы, в которых лежит информация о том, как показывалась реклама пользователям и как они на нее кликали (или не кликали).

Сами данные будет подготавливать в спарке, а обучать машину будем через VW. 
**Для семинара** - данные лучше загружать заранее. Для удобства, они выложены в открытый доступ с Azure.

```
https://hadoop2hdistorage2.blob.core.windows.net/spark2-2020-02-09t17-13-49-527z/avito/AdsInfo.tsv
https://hadoop2hdistorage2.blob.core.windows.net/spark2-2020-02-09t17-13-49-527z/avito/Category.tsv
https://hadoop2hdistorage2.blob.core.windows.net/spark2-2020-02-09t17-13-49-527z/avito/Location.tsv
https://hadoop2hdistorage2.blob.core.windows.net/spark2-2020-02-09t17-13-49-527z/avito/PhoneRequestsStream.tsv
https://hadoop2hdistorage2.blob.core.windows.net/spark2-2020-02-09t17-13-49-527z/avito/SearchInfo.tsv
https://hadoop2hdistorage2.blob.core.windows.net/spark2-2020-02-09t17-13-49-527z/avito/trainSearchStream.tsv
https://hadoop2hdistorage2.blob.core.windows.net/spark2-2020-02-09t17-13-49-527z/avito/UserInfo.tsv
https://hadoop2hdistorage2.blob.core.windows.net/spark2-2020-02-09t17-13-49-527z/avito/VisitsStream.tsv
```

# Сборка кролика

In [ ]:
! git clone --recursive https://github.com/VowpalWabbit/vowpal_wabbit.git
! sudo apt install libboost-dev libboost-program-options-dev libboost-system-dev libboost-thread-dev libboost-math-dev libboost-test-dev zlib1g-dev cmake g++ -y
! cd vowpal_wabbit && make && cd build && make install
! echo "export PATH=/usr/local/bin:\$PATH" >> ~/.bashrc

Или можно просто стащить уже собраный бинарь отсюда - http://finance.yendor.com/ML/VW/Binaries/

In [ ]:
! mkdir -p bin/
! wget http://finance.yendor.com/ML/VW/Binaries/vw-8.20190624 -O bin/vw
! chmod +x bin/vw

In [16]:
# ! echo "export PATH=$(pwd)/bin:\$PATH" >> ~/.bashrc

# Загрузим датасет

https://scikit-learn.org/0.18/datasets/rcv1.html

Reuters Corpus Volume I (RCV1) is an archive of over 800,000 manually categorized newswire stories made available by Reuters, Ltd. for research purposes. 

In [2]:
! wget http://hunch.net/~vw/rcv1.tar.gz -O rcv1.tar.gz

--2019-12-25 11:30:57--  http://hunch.net/~vw/rcv1.tar.gz
Resolving hunch.net (hunch.net)... 188.138.121.136
Connecting to hunch.net (hunch.net)|188.138.121.136|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456931808 (436M) [application/x-gzip]
Saving to: ‘rcv1.tar.gz’

rcv1.tar.gz         100%[===================>] 435.76M   755KB/s    in 14m 57s 

2019-12-25 11:45:54 (498 KB/s) - ‘rcv1.tar.gz’ saved [456931808/456931808]



In [3]:
! tar -zxvf rcv1.tar.gz 

rcv1/
rcv1/rcv1.train.vw.gz
rcv1/vw_process
rcv1/rcv1.test.vw.gz


In [4]:
! head rcv1/rcv1.train.vw.gz | gunzip

1 |f 13:3.9656971e-02 24:3.4781646e-02 69:4.6296168e-02 85:6.1853945e-02 140:3.2349996e-02 156:1.0290844e-01 175:6.8493910e-02 188:2.8366476e-02 229:7.4871540e-02 230:9.1505975e-02 234:5.4200061e-02 236:4.4855952e-02 238:5.3422898e-02 387:1.4059304e-01 394:7.5131744e-02 433:1.1118756e-01 434:1.2540409e-01 438:6.5452829e-02 465:2.2644201e-01 468:8.5926279e-02 518:1.0214076e-01 534:9.4191484e-02 613:7.0990764e-02 646:8.7701865e-02 660:7.2289191e-02 709:9.0660661e-02 752:1.0580081e-01 757:6.7965068e-02 812:2.2685185e-01 932:6.8250686e-02 1028:4.8203137e-02 1122:1.2381379e-01 1160:1.3038123e-01 1189:7.1542501e-02 1530:9.2655659e-02 1664:6.5160148e-02 1865:8.5823394e-02 2524:1.6407280e-01 2525:1.1528353e-01 2526:9.7131468e-02 2536:5.7415009e-01 2543:1.4978983e-01 2848:1.0446861e-01 3370:9.2423186e-02 3960:1.5554591e-01 7052:1.2632671e-01 16893:1.9762035e-01 24036:3.2674628e-01 24303:2.2660980e-01
0 |f 9:8.5609287e-02 14:2.9904654e-02 19:6.1031535e-02 20:2.1757640e-02 24:1.3484491e-02 39:5.0

# Формат

```
[Label] [Importance] [Base] [Tag]|Namespace Features |Namespace Features ... |Namespace Features
```

* Label - значение целевой переменной. Если не указывать, объект не будет использоваться в обучении
* Importance - вес объекта. Если не указывать, равен 1
* Base - "смещение" при предсказании - см. residual regression . Если не указывать, равен 0
* Tag - пометка объекта. Никак не влияет на процесс обучения, но добавляет семантики и "читаемости" данных для человека
* Namespace - название области признаков. Используется, чтобы разные по сути фичи с одинаковым названием не пересекались
* Features - фичи. Это или пара <название фичи>:<значение> или просто <название фичи>. В последнем случае будет считаться, что значение равно 1. 

# Запуск

In [5]:
! time vw rcv1/rcv1.train.vw.gz --final_regressor result.vw.bin

final_regressor = result.vw.bin
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = rcv1/rcv1.train.vw.gz
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.0000       50
0.511062 0.022124            2            2.0   0.0000   0.1487      103
0.260481 0.009900            4            4.0   0.0000   0.0418      134
0.237231 0.213981            8            8.0   0.0000   0.1846      145
0.247655 0.258079           16           16.0   1.0000   0.2879       23
0.242766 0.237877           32           32.0   0.0000   0.1818       31
0.236583 0.230399           64           64.0   0.0000   0.1566       60
0.229860 0.223137          128          128.0   1.0000   0.8289      105
0.186476 0.143092          256          256.0   1.0000   0.8526      122
0.156768 0.127060       

* `-f`, `--final_regressor` - куда сложить результат

In [6]:
! du -h result.vw.bin

308K	result.vw.bin


In [7]:
! time vw --testonly --initial_regressor result.vw.bin --predictions predictions.txt rcv1/rcv1.test.vw.gz

only testing
predictions = predictions.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = rcv1/rcv1.test.vw.gz
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0   0.0000   0.0000       40
0.000000 0.000000            2            2.0   1.0000   1.0000       74
0.000000 0.000000            4            4.0   0.0000   0.0000       43
0.000000 0.000000            8            8.0   0.0000   0.0000       41
0.007352 0.014703           16           16.0   1.0000   1.0000       33
0.006755 0.006159           32           32.0   0.0000   0.0000      209
0.017643 0.028531           64           64.0   1.0000   1.0000       24
0.019877 0.022112          128          128.0   1.0000   1.0000       44
0.066147 0.112417          256          256.0   0.0000   0.0000       48
0.056477 0.046

* `-t`, `--testonly` - не обучать ничего, только предикты
* `-i`, `--initial_regressor` - использовать начальные веса из файла
* `-p`, `--predictions` - куда сложить предсказания

In [8]:
! head predictions.txt

0
1
0
0
0
0
0
0
0
0.318022


# Категориальные фичи

Категориальные фичи предполагается кодировать различными бинарными признаками

In [9]:
import os

os.system("""cat <<EOT >> cat_feat.vw
1 | color_1 color_2 enabled
1 | color_3 endabled
0 | color_1 disabled
1 | color_2 color_3 disabled
0 | color_2 disabled
EOT""")

0

In [10]:
! cat cat_feat.vw

1 | color_1 color_2 enabled
1 | color_3 endabled
0 | color_1 disabled
1 | color_2 color_3 disabled
0 | color_2 disabled


In [11]:
! vw cat_feat.vw -f cat_result.vw.bin

final_regressor = cat_result.vw.bin
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = cat_feat.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.0000        4
0.854476 0.708951            2            2.0   1.0000   0.1580        3
0.565922 0.277369            4            4.0   1.0000   0.3936        4

finished run
number of examples = 5
weighted example sum = 5.000000
weighted label sum = 3.000000
average loss = 0.503381
best constant = 0.600000
best constant's loss = 0.240000
total feature number = 17


In [12]:
! vw -t -i cat_result.vw.bin -p cat_preds.txt cat_feat.vw

only testing
predictions = cat_preds.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = cat_feat.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.126729 0.126729            1            1.0   1.0000   0.6440        4
0.112434 0.098139            2            2.0   1.0000   0.6867        3
0.118843 0.125252            4            4.0   1.0000   0.5322        4

finished run
number of examples = 5
weighted example sum = 5.000000
weighted label sum = 3.000000
average loss = 0.107927
best constant = 0.600000
best constant's loss = 0.240000
total feature number = 17


In [13]:
! cat cat_preds.txt

0.644010
0.686728
0.177915
0.532185
0.253504


# Докрутки

In [14]:
! vw --help

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1
driver:
  --onethread           Disable parse thread
VW options:
  --ring_size arg (=256, ) size of example ring
  --strict_parse           throw on malformed examples
Update options:
  -l [ --learning_rate ] arg Set learning rate
  --power_t arg              t power value
  --decay_learning_rate arg  Set Decay factor for learning_rate between passes
  --initial_t arg            initial t value
  --feature_mask arg         Use existing regressor to determine which 
                             parameters may be updated.  If no 
                             initial_regressor given, also used for initial 
                             weights.
Weight options:
  -i [ --initial_regressor ] arg  Initial regressor(s)
  --initial_weight arg            Set all weights to an initial value of arg.
  --random_weights                make initial weights random
  --normal_weights  

In [ ]:
! vw --normal_weights --quadratic ff --loss_function logistic --l2 0.1 rcv1/rcv1.train.vw.gz --final_regressor result2.vw.bin

Note: Задачи классификации требуют значения целевой переменной из набора {-1, 1}

In [16]:
! cat rcv1/rcv1.train.vw.gz | gunzip | sed -e 's/^0/-1/' > logistic.train.vw

In [17]:
! cat rcv1/rcv1.test.vw.gz | gunzip | sed -e 's/^0/-1/' > logistic.test.vw

In [18]:
! head logistic.train.vw

1 |f 13:3.9656971e-02 24:3.4781646e-02 69:4.6296168e-02 85:6.1853945e-02 140:3.2349996e-02 156:1.0290844e-01 175:6.8493910e-02 188:2.8366476e-02 229:7.4871540e-02 230:9.1505975e-02 234:5.4200061e-02 236:4.4855952e-02 238:5.3422898e-02 387:1.4059304e-01 394:7.5131744e-02 433:1.1118756e-01 434:1.2540409e-01 438:6.5452829e-02 465:2.2644201e-01 468:8.5926279e-02 518:1.0214076e-01 534:9.4191484e-02 613:7.0990764e-02 646:8.7701865e-02 660:7.2289191e-02 709:9.0660661e-02 752:1.0580081e-01 757:6.7965068e-02 812:2.2685185e-01 932:6.8250686e-02 1028:4.8203137e-02 1122:1.2381379e-01 1160:1.3038123e-01 1189:7.1542501e-02 1530:9.2655659e-02 1664:6.5160148e-02 1865:8.5823394e-02 2524:1.6407280e-01 2525:1.1528353e-01 2526:9.7131468e-02 2536:5.7415009e-01 2543:1.4978983e-01 2848:1.0446861e-01 3370:9.2423186e-02 3960:1.5554591e-01 7052:1.2632671e-01 16893:1.9762035e-01 24036:3.2674628e-01 24303:2.2660980e-01
-1 |f 9:8.5609287e-02 14:2.9904654e-02 19:6.1031535e-02 20:2.1757640e-02 24:1.3484491e-02 39:5.

In [19]:
! time vw --normal_weights --quadratic ff --loss_function logistic --l2 0.1 logistic.train.vw --final_regressor result2.vw.bin

creating quadratic features for pairs: ff 
using l2 regularization = 0.1
final_regressor = result2.vw.bin
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = logistic.train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.719974 0.719974            1            1.0   1.0000  -0.0530     1275
0.908148 1.096321            2            2.0  -1.0000   0.6897     5356
0.746713 0.585279            4            4.0  -1.0000   0.2299     9045
0.667753 0.588792            8            8.0  -1.0000  -1.9259    10585
0.558648 0.449542           16           16.0   1.0000  -0.0777      276
0.654712 0.750777           32           32.0  -1.0000  -0.2246      496
0.681594 0.708476           64           64.0  -1.0000  -0.5785     1830
0.654111 0.626627          128          128.0   1.0000  -0.5068     5565
0.647276 0.640442          

Note: даже с квадратичным размером оригинальный фичей, кролик не загнулся, а справился благодаря хешированию

In [20]:
! vw logistic.test.vw -t -i result2.vw.bin --loss_function=logistic --link=logistic -p preds2.txt 

creating quadratic features for pairs: ff 
only testing
predictions = preds2.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = logistic.test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.989140 0.989140            1            1.0  -1.0000  -0.0054      820
0.999102 1.009063            2            2.0   1.0000  -0.0045     2775
0.993647 0.988192            4            4.0  -1.0000  -0.0053      946
0.987501 0.981356            8            8.0  -1.0000  -0.0063      861
0.989760 0.992019           16           16.0   1.0000  -0.0014      561
0.988507 0.987254           32           32.0  -1.0000  -0.0308    21945
0.995975 1.003442           64           64.0   1.0000  -0.0033      300
0.996335 0.996696          128          128.0   1.0000  -0.0027      990
0.995209 0.994083          256          256.0  -1.000

In [21]:
! head preds2.txt

-0.005445
-0.004521
-0.006548
-0.005296
-0.006130
-0.016499
-0.008593
-0.006277
-0.007001
-0.012910


# Сбор датасета для обучения VW

Будем составлять датасет, основываясь на данных в предоставленных таблицах от Авито (смотри начало ноутбука). 
Таблицы с исходными данными достаточно большие, поэтому будем использовать Spark для предобработки и подготовки датасета.

__Напоминалка__: Необходимо подключиться к головной машине через ssh, открыть файл `/usr/bin/anaconda/lib/python2.7/site-packages/nbformat/_version.py` и заменить 5 на 4. После этого остается перезагрузить Jupyter через ambari.

Распарсим таблицы в HDFS и зарегистрируем таблицы, чтобы можно было использовать Spark SQL

In [ ]:
locationDf = spark.read.option("delimiter", "\t").csv('/avito/Location.tsv', header=True, inferSchema=True)
adsInfoDf = spark.read.option("delimiter", "\t").csv('/avito/AdsInfo.tsv', header=True, inferSchema=True)
categoryDf = spark.read.option("delimiter", "\t").csv('/avito/Category.tsv', header=True, inferSchema=True)
phoneRequestStreamDf = spark.read.option("delimiter", "\t").csv('/avito/PhoneRequestsStream.tsv', header=True, inferSchema=True)
searchInfoDf = spark.read.option("delimiter", "\t").csv('/avito/SearchInfo.tsv', header=True, inferSchema=True)
visitStreamDf = spark.read.option("delimiter", "\t").csv('/avito/VisitsStream.tsv', header=True, inferSchema=True)
userInfoDf = spark.read.option("delimiter", "\t").csv('/avito/UserInfo.tsv', header=True, inferSchema=True)
searchStreamDf = spark.read.option("delimiter", "\t").csv('/avito/trainSearchStream.tsv', header=True, inferSchema=True)

In [ ]:
locationDf.registerTempTable('location')
adsInfoDf.registerTempTable('ads_info')
categoryDf.registerTempTable('category')
phoneRequestStreamDf.registerTempTable('phone_request_stream')
searchInfoDf.registerTempTable('search_info')
visitStreamDf.registerTempTable('visit_stream')
userInfoDf.registerTempTable('user_info')
searchStreamDf.registerTempTable('search_stream')

In [ ]:
r = spark.sql("""
select count(*) from search_stream
""")
r.show()

Отлично! Теперь мы осталось взглянуть на данные и попробовать собрать что-то из таблиц.

Для самого простого примера возьмем все контекстуальные показы рекламы и возьмем для них два вещественных признака - HistCTR и Price. Так как информация по цене находится в другой таблице, нам потребуется приджойнить ее к основной таблице.

In [ ]:
result  = spark.sql("""
select s.IsClick, s.HistCTR, ai.price
from search_stream s
join ads_info ai on s.AdID = ai.AdID
where ai.IsContext = True
""")

Осталось привести это в нужный формат и можно сохранять

In [ ]:
def make_vw(row):
    return "{label} |f hist:{hist} price:{price}".format(
        label=1 if row.IsClick == 1 else -1, 
        hist=row.HistCTR, 
        price=row.price or 0
    )

result.rdd.map(make_vw).saveAsTextFile("wasb:///avito/dataset-v1.data")

После завершения задачи, перебираемся на головную машину и выкачиваем получившийся датасет.

In [ ]:
! hdfs dfs -getmerge /avito/dataset-v1.data dataset-v1.data

Дополнительно разделим на train/test

In [ ]:
! cat dataset-v1.data | wc -l
! head -n 100000 dataset-v1.data > test-v1.data
! tail -n +100000 dataset-v1.data > train-v1.data

Можно запускать VW

In [ ]:
! vw train-v1.data --loss_function=logistic --link=logistic --final_regressor result-v1.vw.bin

In [ ]:
! vw --testonly --initial_regressor result-v1.vw.bin --loss_function=logistic --link=logistic --predictions predictions-v1.txt test-v1.data

Давайте накинем туда еще каких-нибудь категориальных и BOW от поискового текста.

In [ ]:
import re

resul = spark.sql("""
select s.IsClick, s.HistCTR, ai.Price, si.SearchQuery, si.IsUserLoggedOn, ui.UserDeviceID, ui.UserAgentOSID
from search_stream s
join ads_info ai on s.AdID = ai.AdID
join search_info si on si.SearchID = s.SearchID
join user_info ui on ui.UserID = si.UserID
where ai.IsContext = True
""")

def make_vw(row):
    return "{label} |f hist:{hist} price:{price} {categr}".format(
        label=1 if row.IsClick == 1 else -1, 
        hist="{0:.5f}".format(row.HistCTR), 
        price=row.Price or 0,
        categr=" ".join([
                "UserLoggedOn" if row.IsUserLoggedOn else "UserLoggedOff",
                "u{}".format(row.UserDeviceID), 
                *re.findall(r'[a-zA-Z]+', (row.SearchQuery or ""))
            ]
        )
    )

resul.rdd.map(make_vw).saveAsTextFile("wasb:///avito/dataset-v2.data")

Прогоним VW на этом

In [ ]:
! hdfs dfs -getmerge /avito/dataset-v2.data dataset-v2.data
! head -n 100000 dataset-v2.data > test-v2.data
! tail -n +100000 dataset-v2.data > train-v2.data
! vw train-v2.data --loss_function=logistic --link=logistic --final_regressor result-v2.vw.bin


In [ ]:
! vw --testonly --initial_regressor result-v2.vw.bin --loss_function=logistic --link=logistic --predictions predictions-v2.txt test-v2.data

**Задачи**

Note: не забудьте, что существует такая классная штука, как Spark 

* Посчитать самые популярные слова в запросах (SearchQuery)
* Подсчитать количество объявлений для каждой категории второго уровня (level = 2) (категории имеют иерархическую структуру - все объявления в подкатегориях (3 уровень) также должны считаться объявлением в соответствующей категории второго уровня)
* Для каждого слова из заголовка объявления подсчитать его среднюю стоимость (если слово появилось в заголовке рекламы с ценой A и в заголовке рекламы с ценой B, то его средняя стоимость - (A+B)/2)
* Найти самый популярный фильтр (ключ в словаре SearchParams)
* Топ 5 слов в пользовательских запросах, в которых пользователь кликнул на рекламу

* Собрать датасет, состоящий из не менее 8 различных фичей (логически различных - в последнем примере фичей было 5 - histCRT, price, user_login, user_device, search_query_bow)